In [116]:
import spacy
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns

#### Data preparation

In [2]:
HCHS_data=pd.read_excel('/workspaces/de.uke.iam.automapping/experiments/VM_Soarian_HCHS_20210422.xlsx')
print(f"Data length {len(HCHS_data['Langname']) } and length of unique data {len(HCHS_data['Langname'].unique())}")

Data length 1033 and length of unique data 959


In [3]:
HCHS_data['Langname']=HCHS_data['Langname'].replace([' ', ''], np.nan)
HCHS_Langname=HCHS_data['Langname'].str.strip()
HCHS_Langname=HCHS_Langname.dropna(axis=0)

In [4]:
HCHS_Langname=HCHS_Langname.unique()


In [5]:
HCHS_Langname

array(['Liegt bei Ihnen eine vom Arzt diagnostizierte Demenz vor?',
       'Liegt bei Ihnen ein vom Arzt diagnostizierter Diabetes vor?',
       'Wann war die letzte Injektion?',
       'Hatten Sie bereits einen vom Arzt diagnostizierten Herzinfarkt?',
       'Ist bei Ihnen eine vom Arzt diagnostizierte Herzinsuffizienz bekannt?',
       'Ist bei Ihnen ein vom Arzt diagnostizierter Hypertonus bekannt?',
       'Ist bei Ihnen eine vom Arzt diagnostizierte KHK bekannt?',
       'Rauchen Sie?',
       'Wurde bei Ihnen jemals ein Schlaganfall von einem Arzt diagnostiziert?',
       'Wurde bei Ihnen jemals ein Vorhofflimmern von einem Arzt diagnostiziert?',
       'Ist bei Ihnen jemals von einem Arzt eine Multiple Sklerose (MS) diagnostiziert worden?',
       'Ist bei Ihnen jemals von einem Arzt ein Hirntumor diagnostiziert worden?',
       'Wird die Medikamentenerfassung durchgeführt?',
       'Berichtete Körpergröße', 'Berichtetes Körpergewicht',
       'Gemessener Taillenumfang', 'Gemess

#### Check existence

In [6]:
nlp=spacy.load('de_core_news_lg') #load model with german language

In [7]:
def get_existence(data):
    """Get words which exist and non exist in spaCy german model"""
    existing_words=[] 
    non_existing_words=[]
    for s in data:
        doc=nlp(s)
        for token in doc:
            if ((token.pos_!='SPACE') & (token.pos_!='CCONJ') & (token.pos_!='PUNCT') & (token.pos_!='NUM')) and not token.is_stop and not token.is_punct: #drop punctuation, number, spaces, stop words
                if token.has_vector is True:
                    existing_words.append(str(token))
                else:
                    non_existing_words.append(str(token))
    return existing_words, non_existing_words



In [8]:
existing_words, non_existing_words=get_existence(HCHS_Langname)

In [9]:
dict_existing_words={}
dict_non_existing_words={}
dict_existing_words=Counter(existing_words)
dict_non_existing_words=Counter(non_existing_words)

print(f"number_of_tokens: {len(existing_words)+len(non_existing_words)}, where {len(existing_words)} words we found and {len(non_existing_words)} words we didn't find")
print(f"percent of existing words: {len(existing_words)/(len(existing_words)+len(non_existing_words)):.1%}")
print(f"Most common words from non existing ones: {dict_non_existing_words.most_common(10)}")
print(f"Most common words from existing ones: {dict_existing_words.most_common(10)}")


number_of_tokens: 2784, where 2180 words we found and 604 words we didn't find
percent of existing words: 78.3%
Most common words from non existing ones: [('extracraniell', 20), ('intracraniell', 20), ('T1_nativ', 17), ('T1_postKM', 17), ('L-G-E', 16), ('Echogenität', 16), ('Stenosegrad', 15), ('abhängigem', 15), ('Nebenbefund', 14), ('Diffusionsrestriktion', 12)]
Most common words from existing ones: [('Segment', 96), ('links', 67), ('rechts', 65), ('Stenose', 64), ('Diameter', 55), ('Mapping', 52), ('Zeile', 37), ('PD', 32), ('vorhanden', 31), ('Plaque', 30)]


#### Playing around spaCy and data 

In [23]:
docs=nlp.pipe(HCHS_Langname)

In [24]:
def extract_tokens_plus_meta(doc:spacy.tokens.doc.Doc):
    """Extract tokens and metadata from individual spaCy doc."""
    return [
        (i.text, i.i, i.lemma_, i.ent_type_, i.tag_, 
         i.dep_, i.pos_, i.is_stop, i.is_alpha, 
         i.is_digit, i.is_punct, i.has_vector) for i in doc
    ]

In [25]:
def tidy_tokens(docs):
    """Extract tokens and metadata from list of spaCy docs."""
    
    cols = [
        "doc_id", "token", "token_order", "lemma", 
        "ent_type", "tag", "dep", "pos", "is_stop", 
        "is_alpha", "is_digit", "is_punct", "has_vector"
    ]
    
    meta_df = []
    for ix, doc in enumerate(docs):
        meta = extract_tokens_plus_meta(doc)
        meta = pd.DataFrame(meta)
        #print(meta)
        meta.columns = cols[1:]
        meta = meta.assign(doc_id = ix).loc[:, cols]
        meta_df.append(meta)
        
    return pd.concat(meta_df)   

In [26]:
df=tidy_tokens(docs)
df.head(5)

,doc_id,token,token_order,lemma,ent_type,tag,dep,pos,is_stop,is_alpha,is_digit,is_punct,has_vector
0,0,Liegt,0,Liegt,,VVFIN,ROOT,VERB,False,True,False,False,True
1,0,bei,1,bei,,APPR,mo,ADP,True,True,False,False,True
2,0,Ihnen,2,ich,,PPER,nk,PRON,True,True,False,False,True
3,0,eine,3,einen,,ART,nk,DET,True,True,False,False,True
4,0,vom,4,vom,,APPRART,sbp,ADP,True,True,False,False,True
